<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#大厂实践" data-toc-modified-id="大厂实践-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>大厂实践</a></span><ul class="toc-item"><li><span><a href="#PPNet" data-toc-modified-id="PPNet-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>PPNet</a></span></li><li><span><a href="#模型拆塔" data-toc-modified-id="模型拆塔-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>模型拆塔</a></span></li></ul></li></ul></div>

# 大厂实践

## PPNet
> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> 1. [PPNET 详解与应用](https://zhuanlan.zhihu.com/p/635364011)

引入门控机制，根据用户人群特征，学习门控参数，提高网络参数个性化和人群化。

PPNet（Parameter Personalized Net），增加 DNN 网络参数个性化。uid，pid，aid 分别表示 user id，photo id，author id。左侧的所有特征的 embedding 会同这 3 个 id 特征的 embedding 拼接到一起作为所有 Gate NN 的输入，左侧所有特征的 embedding 并不接受 Gate NN 的反传梯度，这样操作的目的是减少 Gate NN 对现有特征 embedding 收敛产生的影响。

GateNN的数量同左侧神经网络的层数一致，GateNN的输出与DNN每一层的输入做element-wise product，GateNN是一个两层的神经网络，其中第二层的激活函数是2 * sigmoid，默认值是1，左侧所有特征的embedding不接受GateNN的反传梯度。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240310180038447.png" alt="image-20240310180038447" style="zoom:50%; display: block; margin: auto;" />

## 模型拆塔
推荐系统精排模型经常存在训练样本不均衡现象，eg：尾部5%用户仅提供了2%样本，而头部3%用户贡献了系统60%的样本，模型权重被少量高频重度用户主导，不利于低活用户推荐效果。可以通过用户分群多塔训练方式优化模型预估精度，并结合精排排序机制迭代，整体改善各活跃度用户的推荐体验。

以下模型为三塔模型，预估用户单次播放短点率、完成率与完播率。本次迭代将完成率目标训练方式升级为finetune network，进而区分不同活跃度用户群体，提升预估精度。

其中，finetune network结构借鉴迁移学习的方式，使用一个主塔训练全量样本，4个分塔训练不同活跃度的用户样本，并用于线上预估对应活跃度分群用户的完成率。主塔天级别同步小模型参数至各个分塔，保证各个分塔学习到全量用户的信号。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20240307123820236.png" alt="image-20240307123820236" style="zoom: 33%; display: block; margin: auto;" />